In [6]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [7]:
def delta_gamma_loss(Delta, Gamma, dS) :
    V_change = dS * Delta + 0.5 * Gamma * dS**2
    return V_change

def Delta_normal_VaR_Daily(Delta, S, Daily_sigma, ret= 0 ,sign_level = 0.01) :
    VaR = norm.ppf(1-sign_level) * Delta * Daily_sigma * S
    return VaR

In [8]:
Delta = -500 # 포지션조심
Gamma = -40 
dS = -5 # 부호조심
delta_gamma_loss (Delta, Gamma, dS)

2000.0

In [9]:
Delta = 5000
S = 40
Daily_sigma = 0.4/np.sqrt(252)
ret = 0
Delta_normal_VaR_Daily(Delta, S, Daily_sigma, ret , sign_level = 0.05)

8289.28312384039

In [93]:
x = np.random.normal(0,10,20000)
V = 120 - 0.01 * (100 * (1+x)-100)**2

In [94]:
pd.Series(V - 100).quantile(0.05)

-38790.63035896029

\begin{align}
\\
\ x = \alpha - u \
\\
\\
\ P(X>\alpha)= P(X>u)(1+ \epsilon  \frac{x}{\beta})^{- \frac{1}{\epsilon}}\\
\\
\ = EVT(u, P(X>u) , \beta, \epsilon, \alpha) \
\end{align}


In [10]:
def EVT(u, p_u, beta, psi, alpha) :
    x = alpha - u
    conditional_P = (1 + psi * x/beta)**(-1/psi)
    return p_u * conditional_P

In [11]:
u = 0.04
p_u = 0.0502
beta = 0.019397
psi = -0.05186
alpha = 0.09
EVT(u, p_u, beta, psi, alpha)

0.0031547891618699214

 # Gaussian Copula Default Time

In [12]:
def Default_Time(lamb, Z) :
    tau = -np.log(1-norm.cdf(Z))/lamb
    return tau

In [13]:
lamb = 0.04
Z = -1.34
Default_Time(lamb, Z)

2.3611373366057737

 # 2 Year Transition Prob


In [14]:
A = (np.array([0.04, 2.04, 87.19, 5.30, 0.4, 0.16, 0.03])/100).reshape(1,-1)
B = (np.array([0.06,0.54,5.3,84.28,5.3,0.26,0.34])/100).reshape(-1,1)
A_to_B = A.dot(B)

 # Barrier Model

In [15]:
m = 2.3263 # distance_to_default
alpha = 0 # (mu - r(Debt growth rate) - 0.5 * sigma^2 )
u = 1 # Maturity
def Prob_Default(m, alpha, u) :
    P_Default = norm.cdf((-m-alpha*u)/np.sqrt(u)) + np.exp(-2*alpha*m) * norm.cdf((-m+alpha*u)/np.sqrt(u))
    return P_Default

In [21]:
Prob_Default(1.96, 0, 1)

0.04999579029644087

In [2]:
(0.00303 - 0.03**2)/(0.03 * 0.97) * 0.5 + 0.5 * (-0.03**2)/(0.03*0.97)

0.021134020618556702

 # KMV Structural Model

In [25]:
d = 2.054 # distance to default
PD = norm.cdf(-d)

In [26]:
PD

0.019987845855041885

In [48]:
norm.cdf(-2.326)

0.010009275340867669

 # 추가 Chava Jarrow

\begin{align}
\\
\ 1 factor => P = \frac{1}{1+exp(-\alpha - \beta X)} \
\\\\
\ 2 factor => P= \frac{1}{1+exp(-\alpha - \beta_1 X_1 - \beta_2 X_2)}
\\
\\
\end{align}

In [77]:
def Prob(params, variable) :
    return 1/(1+np.exp( (params * -np.concatenate([[1],variable])).sum() ))
def Marginal_Prob(params, variable) :
    P = 1/(1+np.exp( (params * -np.concatenate([[1],variable])).sum() ))
    M = (1/P)**2 * np.exp((params * -np.concatenate([[1],variable])).sum() ) * np.array(params)[1:].reshape(-1,1)
    return M.reshape(-1)

In [99]:
# one factor
Prob([-1.6406, 0.6803], [1])

0.27681813389831134

In [101]:
# two factor
Prob([-1.6406, -0.1667, 0.6803], [2, .5])

0.16330363219489724

\begin{align}
\\
\ 1 factor => \frac{dP}{dX} = {(\frac{1}{P})}^2 \bullet {exp(-\alpha - \beta X)} \beta \
\\\\
\ 2 factor => \frac{dP}{dX_1} = {(\frac{1}{P})}^2 \bullet {exp(-\alpha - \beta_1 X_1 - \beta_2 X_2 )} \beta_1 \\ \ \frac{dP}{dX_2} = {(\frac{1}{P})}^2 \bullet {exp(-\alpha - \beta_1 X_1 - \beta_2 X_2 )} \beta_2 \ 
\\
\\
\end{align}

In [103]:
# one factor
Marginal_Prob([-1.6406, 0.6803], [.8])

array([32.47107974])

In [102]:
# two factor
Marginal_Prob([-1.6406, -0.1667, 0.6803], [2,0.5])

array([-32.02697717, 130.70157512])

 # Weighted historical simulation

\begin{align}
\\
\ w_N = \frac{(1-\lambda)}{(1-\lambda^{number of data})} \lambda^{N-1}\
\\
\\
\end{align}


In [91]:
number_of_data = 500
lamb = 0.95
nth_past = 5

In [92]:
w = (1-lamb)/(1-lamb**number_of_data) * lamb**(nth_past-1)

In [93]:
w

0.04072531250029628